In [ ]:
import jtrace
import re
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
hsc = """
#
# Column 0: Name
# Column 1: Type
# Column 2: Curvature R (mm)
# Column 3: Thickness dz (mm)
# Column 4: Outer Radius (mm)
# Column 5: Inner Radius (mm)
# Column 6: Conic Constant Kappa
# Column 7 - 20: Aspheric Coefficient a_3 - a_16 (a_n r^n in meters)
# Column 15: Coating file
# Column 16: Medium file
# (0)    (1)               (2)                (3)            (4) (5)      (6) (7) (8)                 (9) (10)                (11) (12)                (13) (14)                (15)                 (16) (17) (18)                (19) (20)                (21) (22)
PM    mirror 30000.0             0.0              4100.0         0.0 -1.00835 0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G1    lens     760.0             13456.4          410.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
G1E   lens    1375.5             97.18            402.20143728   0.0 0.0      0.0 -1.497488783073E-10 0.0  -7.579068214673E-17 0.0  -7.705293869522E-22 0.0  1.026626319251E-26  0.0  -7.078826218129E-32  0.0  2.559600617806E-37  0.0  -3.761551480578E-43 ---  air
G2    lens   -3530.0             372.85           307.5181146466 0.0 0.0      0.0 6.914718060975E-11  0.0  6.03728126364E-16   0.0  -1.532623173462E-20 0.0  3.618110894954E-25  0.0  -4.741677881169E-30  0.0  3.217309071692E-35  0.0  -8.85987809163E-41  ---  bsl7y
G2E   lens   656.2498974113356   46.0             286.5165285897 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
ADC1  lens   0.0                 318.0            282.794660157  0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  bsl7y
ADC1E lens   1058.0              40.0             282.0282677958 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
ADC2  lens   1040.0              3.0              282.5217682803 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  pbl1y
ADC2E lens   0.0                 82.0             281.8065578377 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G3    lens   -840.000170144847   274.3            275.8409560694 0.0 0.0      0.0 2.768476102307E-9   0.0  -4.855642009259E-14 0.0  7.176113272766E-19  0.0  -1.076370261452E-23 0.0  1.18744321781E-28    0.0  -7.983821197395E-34 0.0  2.393556961408E-39  ---  pbl1y
G3E   lens   9800.0              40.0             283.8358237172 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G4    lens   480.0               90.0             314.0          0.0 0.0      0.0 -4.355533915492E-9  0.0  3.635867914913E-14  0.0  -5.951267151922E-19 0.0  7.658839741989E-24  0.0  -7.194120343205E-29  0.0  3.942820924898E-34  0.0  -9.54342843455E-40  ---  bsl7y
G4E   lens   4021.7590211374127  102.0            314.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G5    lens   4176.748363640779   100.2            314.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
G5E   lens   -1272.8222984309575 88.0             308.0          0.0 0.0      0.0 -1.064687616564E-9  0.0  3.377750239208E-15  0.0  -1.1026471335E-19   0.0  2.282368582236E-24  0.0  -2.743039658171E-29  0.0  1.755771152898E-34  0.0  -4.821949608578E-40 ---  air
F     filter 0.0                 90.04125645151   269.5068874933 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
FE    filter 0.0                 15.0             266.8394944827 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
W     lens   0.0                 32.5             258.2205358629 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
WE    lens   0.0                 37.0             251.6409664366 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
D     det    0.0                 15.0             247.6837392465 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  ---
"""
re.sub("\s+"," ", hsc)
None

In [ ]:
bsl7y = jtrace.ConstMedium(1.50939738)
silica = jtrace.ConstMedium(1.45172729)
pbl1y = jtrace.ConstMedium(1.53789058)
air = jtrace.ConstMedium(1.0)

In [ ]:
telescope = []
focus = 0.0
z = 0.0
m0 = air
m1 = air
for line in hsc.split('\n'):
    if len(line) == 0 : continue
    if line[0] == '#': continue
    name, typ, R, dz, outer, inner, kappa, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, coating, medium = line.split()
    if name == 'G1':
        dz = float(dz) + focus
    z += float(dz)/1000
    if typ == 'none': continue
    m0 = m1
    if medium == 'air': 
        m1 = air
    elif medium == 'pbl1y':
        m1 = pbl1y
    elif medium == 'bsl7y':
        m1 = bsl7y
    elif medium == 'silica':
        m1 = silica
    elif medium == '---':
        pass
    else:
        raise ValueError("Unknown medium {}".format(medium))
    if float(R) == 0:
        surface = jtrace.Plane(z)
    else:
        # Notice the manipulation of the raw aspheric coefficients below.  There's a negative sign missing, 
        # and also the coefficients are for mm instead of m, so there are factors of 1000^4, ^6, and ^8 missing
        # as well.  Actually, the factors or 1000^(4-1), ^(6-1), ^(8-1) for some weird reason.  It's like the
        # zemax crowd just changed the coefficients from m to mm by multiplying by 1000 instead of 1000 to the
        # appropriate power.  Weird.
        alpha = [np.double(a)*10**(3*(i-1)) 
                 for a, i in zip([a4, a6, a8, a10, a12, a14, a16],
                                 [4, 6, 8, 10, 12, 14, 16])]
        if all(a == 0.0 for a in alpha):
            surface = jtrace.Quadric(float(R)/1000, float(kappa), z)
        else:
            surface = jtrace.Asphere(float(R)/1000, float(kappa), alpha, z)
    telescope.append(dict(name=name, surface=surface, outer=float(outer)/1000, inner=float(inner)/1000, m0=m0, m1=m1, typ=typ))

In [ ]:
for optic in telescope:
    print(optic['name'], type(optic['surface']))

In [ ]:
plt.figure(figsize=(12, 11))
for optic in telescope:
    x = np.linspace(-optic['outer'], -optic['inner'], 1000, endpoint=True)
    plt.plot(x, [optic['surface'](x_,0) for x_ in x], c='k')
    x = np.linspace(optic['inner'], optic['outer'], 1000, endpoint=True)
    plt.plot(x, [optic['surface'](x_,0) for x_ in x], c='k')

def trace(ray, telescope, color='r'):
    r = ray
    for optic in telescope:
        x0 = r.x0
        z0 = r.z0
        isec = optic['surface'].intersect(r)
        assert abs(isec.point.z - optic['surface'](isec.point.x, isec.point.y)) < 1e-10
        if optic['typ'] == 'mirror':
            r = isec.reflectedRay(r)
        elif optic['typ'] in ['lens', 'filter']:
            r = isec.refractedRay(r, optic['m0'], optic['m1'])
        x1 = isec.x0
        z1 = isec.z0
        plt.plot([x0, x1], [z0, z1], c=color)
    return isec.point

p0_left = jtrace.Vec3(-3.5, 0, 20)
p0_right = jtrace.Vec3(3.5, 0, 20)
v_left = jtrace.Vec3(-0.004, 0, -1).UnitVec3()
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()
v_right = jtrace.Vec3(0.004, 0, -1).UnitVec3()

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    for p0 in [p0_left, p0_right]:
        ray = jtrace.Ray(p0, v, t=0.0, w=0.8)
        print(trace(ray, telescope, color=c))

In [ ]:
plt.figure(figsize=(12, 11))
for optic in telescope:
    x = np.linspace(-optic['outer'], -optic['inner'], 1000, endpoint=True)
    plt.plot(x, [optic['surface'](x_,0) for x_ in x], c='k')
    x = np.linspace(optic['inner'], optic['outer'], 1000, endpoint=True)
    plt.plot(x, [optic['surface'](x_,0) for x_ in x], c='k')

def trace(ray, telescope, color='r'):
    r = ray
    for optic in telescope:
        x0 = r.x0
        z0 = r.z0
        isec = optic['surface'].intersect(r)
        assert abs(isec.point.z - optic['surface'](isec.point.x, isec.point.y)) < 1e-10
        if optic['typ'] == 'mirror':
            r = isec.reflectedRay(r)
        elif optic['typ'] in ['lens', 'filter']:
            r = isec.refractedRay(r, optic['m0'], optic['m1'])
        x1 = isec.x0
        z1 = isec.z0
        plt.plot([x0, x1], [z0, z1], c=color)
    return isec.point
plt.ylim(13.0, 15.5)
plt.xlim(-0.5, 0.5)

p0_left = jtrace.Vec3(-3.5, 0, 20)
p0_right = jtrace.Vec3(3.5, 0, 20)
v_left = jtrace.Vec3(-0.004, 0, -1).UnitVec3()
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()
v_right = jtrace.Vec3(0.004, 0, -1).UnitVec3()

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    for p0 in [p0_left, p0_right]:
        ray = jtrace.Ray(p0, v, t=0.0, w=0.8)
        print(trace(ray, telescope, color=c))